# Recurrent Neural Networks (RNNs)


**FNNs** employ fixed-size vectors and have difficulties dealing with sequences of varying length: they fail to capture the temporal aspects of language.
**RNNs** solve this problem with a cycle within its network connections:

![image-3.png](img/RNN-unrolled.png)
The activation value of the hidden layer depends on the current input as well as the activation value of the hidden layer from the previous time step.

**Forward inference** is similar to what we saw with FNNs:
$$\begin{align}
   &h_t = g(Uh_{t-1}+Wx_t)\newline
   &y_t=f(Vh_t)
   \end{align}$$
This is an **incremental inference algorithm**, it can be interpreted as unrolling the network in time:
![image.png](img/RNN_time.png)

### Example: Sentiment classification on a moview review dataset
The following example is taken from: https://github.com/Harvard-IACS/2019-CS109B/blob/master/content/labs/lab6/cs109b-lab6-rnn-solutions.ipynb

<ul>
<li>25000 movie reviews from IMDB, labeled either positive or negative</li>
<li>In each review, words are indexed by their overall frequency</li>
</ul>

A sentence can be thought of as a sequence of words which have semantic connections across time.

In [71]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.preprocessing import sequence

# fix random seed for reproducibility
np.random.seed(1)

We define some parameters:

In [47]:
# take the 10000 most common words
VOCABULARY_SIZE = 10000

#Finite length of the review (avoid long sentences)
MAX_REVIEW_LENGTH = 500


EMBEDDING_DIM = 100

Normally, we would proceed by **tokenising** the text, but the IMDB dataset is already tokenised.
Then **load the data**:

In [48]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=VOCABULARY_SIZE)

print("Categories:", np.unique(y_train))
print("Number of unique words:", len(np.unique(np.hstack(X_train))))

print('First review', X_train[0])
print('Length of first and fifth review', len(X_train[0]) ,len(X_train[4]))

Categories: [0 1]
Number of unique words: 9998
First review [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


We now **pre-process data** by ensuring that all inputs have same sentence length and dimensions. We use Keras' `sequence.pad_sequences()` function:

In [49]:
X_train = sequence.pad_sequences(X_train, maxlen=MAX_REVIEW_LENGTH)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_REVIEW_LENGTH)
print('Length of first and fifth review after padding', len(X_train[0]) ,len(X_train[4]))

Length of first and fifth review after padding 500 500


#### FNNs with embeddings

![image.png](img/FNN.png)

We build a single layer FNN with 250 nodes. Each input will be a 500-dim vector of tokens (remember padding).

In [50]:
#instantiate the model. Sequential groups a linear stack of layers into the model.
model_fnn = Sequential()

#add the embedding layer
model_fnn.add(Embedding(VOCABULARY_SIZE, EMBEDDING_DIM, input_length=MAX_REVIEW_LENGTH))

#we flatten the input layer to one dimension
model_fnn.add(Flatten())

#hidden layer
model_fnn.add(Dense(250, activation='relu'))

#output layer 
model_fnn.add(Dense(1, activation='sigmoid'))

#configure model for training
model_fnn.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
print(model_fnn.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
flatten_3 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 250)               12500250  
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 251       
Total params: 13,500,501
Trainable params: 13,500,501
Non-trainable params: 0
_________________________________________________________________
None


Now we fit the model and evaluate its performance:

In [51]:
# Fit the model
model_fnn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128, verbose=2)

# Evaluation of the model
scores = model_fnn.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/2
196/196 - 15s - loss: 0.6931 - accuracy: 0.5051 - val_loss: 0.6929 - val_accuracy: 0.5116
Epoch 2/2
196/196 - 16s - loss: 0.6920 - accuracy: 0.5247 - val_loss: 0.6923 - val_accuracy: 0.5164
Accuracy: 51.64%


### Simple RNN

Similar procedure, but here hidden states are created using the state variable from the previous timestep and the input at current time.

In [81]:
#instantiate the model. Sequential groups a linear stack of layers into the model.
model_rnn = Sequential()

#add the embedding layer
model_rnn.add(Embedding(VOCABULARY_SIZE, EMBEDDING_DIM, input_length=MAX_REVIEW_LENGTH))

#add a fully-connected RNN with 100 units
model_rnn.add(SimpleRNN(100))

#output layer
model_rnn.add(Dense(1, activation='sigmoid'))

#configure model for training
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_rnn.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 500, 100)          1000000   
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 100)               20100     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 101       
Total params: 1,020,201
Trainable params: 1,020,201
Non-trainable params: 0
_________________________________________________________________
None


In [82]:
#fit the model
model_rnn.fit(X_train, y_train, epochs=2, batch_size=128)

# Final evaluation of the model
scores = model_rnn.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/2
196/196 [==============================] - 29s 149ms/step - loss: 0.5553 - accuracy: 0.6896
Epoch 2/2
196/196 [==============================] - 28s 145ms/step - loss: 0.5162 - accuracy: 0.7413
Accuracy: 68.13%


During backpropagation, simple RNNs can have ***vanishing gradients***. In fact, calculating the derivative of the sigmoid:
$$\sigma(z)=\frac{1}{1+e^{-x}}\quad \to \quad \frac{d\sigma}{dz}=\sigma(z)(1-\sigma(z))$$

### LSTMs

![image.png](img/LSTM.png)

Long short-term memory or LSTMS handle the context in a more intuitive way:
<ul>
<li>Remove information no longer needed</li>
<li>Add information likely to be needed lately</li>
</ul>

They introduce different gates: ***forget gate, add gate, output gate*** .

In [54]:
#instantiate the model. Sequential groups a linear stack of layers into the model.
model_lstm = Sequential()

#add the embedding layer
model_lstm.add(Embedding(VOCABULARY_SIZE, EMBEDDING_DIM, input_length=MAX_REVIEW_LENGTH))

#add LSTM layer with output dimensionality 100
model_lstm.add(LSTM(100))

model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
print(model_lstm.summary())

model_lstm.fit(X_train, y_train, epochs=2, batch_size=128)

# Final evaluation of the model
scores = model_lstm.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 101       
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
196/196 [==============================] - 76s 389ms/step - loss: 0.6930 - accuracy: 0.5096
Epoch 2/2
196/196 [==============================] - 79s 405ms/step - loss: 0.6930 - accuracy: 0.5106
Accuracy: 51.45%


Note number of parameters compared to simple RNN.

How to improve the accuracy?

More advanced model (taken from https://www.tensorflow.org/tutorials/text/text_classification_rnn#create_the_model):

![image.png](img/tensorflow.png)

In [80]:
model = Sequential([Embedding(input_dim=VOCABULARY_SIZE,
                              output_dim=64,
                              # Use masking to handle the variable sequence lengths
                              mask_zero=True),
                    Bidirectional(LSTM(64)),
                    Dense(64, activation='relu'),Dense(1)])

Extra stuff (better not to show it..)

In [69]:
def decode(review_index):
    index = imdb.get_word_index()
    reverse_index = dict([(value, key) for (key, value) in index.items()]) 
    decoded = " ".join( [reverse_index.get(i - 3, "#") for i in X_train[review_index]] )
    print(decoded) 


In [57]:
#Good review
decode(449)

print('\n' * 3)

#Bad review
decode(7337)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # here it is the first ever episode of friends where we get introduced to control freak monica # # cox newly divorced ross # david # # # # lisa # unknown actor and ladies man matt le # and very sarcastic chandler bing matthew perry this is how the scene starts off until we introduced to the # and final friend # kid rachel green jennifer aniston br br the episode is better than most people give credit for like any new sitcom the first episode isn't always fantastic the acting in this episode isn't great because the cast cannot identify and # really believable in their new characters apart from # and perry who shine br br matt le # man his acting was down right dreadful because until

In [63]:
text_bad = X_train[7737]
text_good = X_train[449]
texts = (text_bad, text_good)
padded_texts = sequence.pad_sequences(texts, maxlen=MAX_REVIEW_LENGTH, value = 0.0)

In [67]:
predictions_fnn = model_fnn.predict(padded_texts)
predictions_rnn = model_rnn.predict(padded_texts)
predictions_lstm = model_lstm.predict(padded_texts)
print(f'{predictions_fnn} \n {predictions_rnn} \n {predictions_lstm}')

[[0.48101017]
 [0.49473682]] 
 [[0.5246494 ]
 [0.46627226]] 
 [[0.49783504]
 [0.5042565 ]]
